In [1]:
!pip install spacy


In [13]:
!pip install newsapi-python

In [5]:
!python -m spacy download en_core_web_lg

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [14]:
import spacy
import en_core_web_lg
from newsapi.newsapi_client import NewsApiClient

In [15]:
nlp_eng = en_core_web_lg.load()
newsapi = NewsApiClient (api_key='db53c380ff8c41c487b4f973f97deadc')

In [37]:
articles = newsapi.get_everything(q='coronavirus', language='en', 
                                  from_param='2020-10-01', to='2020-10-27', sort_by='relevancy')

In [24]:
articles

{'status': 'ok',
 'totalResults': 159346,
 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': None,
   'title': 'World number one Johnson tests positive for coronavirus',
   'description': 'World number one Dustin Johnson tests positive for coronavirus, the PGA Tour announces.',
   'url': 'https://www.bbc.co.uk/sport/golf/54532968',
   'urlToImage': 'https://ichef.bbci.co.uk/live-experience/cps/624/cpsprodpb/C971/production/_114896515_johnson.jpg',
   'publishedAt': '2020-10-13T20:06:08Z',
   'content': 'Johnson playing in the US Open in September\r\nWorld number one Dustin Johnson has tested positive for coronavirus, the PGA Tour has announced.\r\nJohnson, 36, who is now self-isolating, has withdrawn fr… [+752 chars]'},
  {'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'https://www.facebook.com/bbcnews',
   'title': 'Bihar elections: Crowds throng rallies amid coronavirus fear',
   'description': 'Experts say that flouting coronavirus safety r

In [42]:
data = []


for article in articles['articles']:
    title = article['title']
    desc = article['description']
    content = article['content']
    data.append({'title': title, 'description':desc, 'content':content})
    


{'title': "Check an Airline's Coronavirus Policies Before Booking", 'description': 'Navigating air travel during the COVID-19 pandemic has been tricky. First and foremost, you want to make sure that you’re as safe as possible. On top of that, there are fewer routes and flights available, travel restrictions, and other guidelines that seem to…', 'content': 'Navigating air travel during the COVID-19 pandemic has been tricky. First and foremost, you want to make sure that youre as safe as possible. On top of that, there are fewer routes and flights availa… [+3231 chars]'}


In [45]:
import pandas as pd

df = pd.DataFrame(data)
df = df.dropna()
df.head()

,title,description,content
0,"Sorry, Mouthwash Isn't a Coronavirus Cure",Mouthwash is designed to kill germs in your mo...,Mouthwash is designed to kill germs in your mo...
1,Check an Airline's Coronavirus Policies Before...,Navigating air travel during the COVID-19 pand...,Navigating air travel during the COVID-19 pand...
2,"Twitter flags, limits sharing on Trump tweet a...",Twitter disabled sharing options on a tweet fr...,The tweet violated the platforms rules against...
3,Coronavirus and Home Design,A recent survey of architects reveals that cli...,The coronavirus pandemic is already shaping ho...
4,Coronavirus Briefing: What Happened Today,What the near future may hold,Many leaders most notably President Trump have...


In [57]:
from collections import Counter

results = []

def get_keywords_eng(data):
    words = []
    doc = nlp_eng(data)
    punctuation = [",",".","!","?"]
    pos_tag = ["VERB", "NOUN", "PROPN"]
    for token in doc:
        if (token.text in nlp_eng.Defaults.stop_words or token.text in punctuation):
            continue
        if (token.pos_ in pos_tag):
            words.append(token.text)
    return words
    
for content in df.content.values:
    results.append([('#' + x[0]) for x in Counter(get_keywords_eng(content)).most_common(5)])
    
df['keywords'] = results

df.head()

,title,description,content,keywords
0,"Sorry, Mouthwash Isn't a Coronavirus Cure",Mouthwash is designed to kill germs in your mo...,Mouthwash is designed to kill germs in your mo...,"[#Mouthwash, #designed, #kill, #germs, #mouth]"
1,Check an Airline's Coronavirus Policies Before...,Navigating air travel during the COVID-19 pand...,Navigating air travel during the COVID-19 pand...,"[#Navigating, #air, #travel, #COVID-19, #pande..."
2,"Twitter flags, limits sharing on Trump tweet a...",Twitter disabled sharing options on a tweet fr...,The tweet violated the platforms rules against...,"[#tweet, #sharing, #violated, #platforms, #rules]"
3,Coronavirus and Home Design,A recent survey of architects reveals that cli...,The coronavirus pandemic is already shaping ho...,"[#pandemic, #coronavirus, #shaping, #home, #de..."
4,Coronavirus Briefing: What Happened Today,What the near future may hold,Many leaders most notably President Trump have...,"[#leaders, #President, #Trump, #described, #ar..."
